<a href="https://colab.research.google.com/github/Osw1997/tsl/blob/main/examples/notebooks/replication_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm tsl -r
!git clone https://github.com/Osw1997/tsl.git

rm: cannot remove 'tsl': No such file or directory
Cloning into 'tsl'...
remote: Enumerating objects: 3531, done.
remote: Counting objects: 100% (705/705), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 3531 (delta 557), reused 518 (delta 467), pack-reused 2826
Receiving objects: 100% (3531/3531), 1.55 MiB | 5.58 MiB/s, done.
Resolving deltas: 100% (2407/2407), done.


In [4]:
# import sys
# sys.path.insert(0, '/content')
# sys.path.append('/content')
'/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl'

'/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl'

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install -q git+https://github.com/TorchSpatiotemporal/tsl.git


2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!rm tsl -r
!git clone https://github.com/Osw1997/tsl.git
!pip install ./tsl
# from tsl.datasets import dataobject_prototype

Cloning into 'tsl'...
remote: Enumerating objects: 3588, done.
remote: Counting objects: 100% (762/762), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 3588 (delta 597), reused 518 (delta 467), pack-reused 2826
Receiving objects: 100% (3588/3588), 1.71 MiB | 8.42 MiB/s, done.
Resolving deltas: 100% (2447/2447), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./tsl
  Preparing metadata (setup.py) ... done
  Created wheel for torch-spatiotemporal: filename=torch_spatiotemporal-0.9.2-py3-none-any.whl size=237936 sha256=f1757c877e854c0af8eabae7117dd790d63a5ce9097471b8b9a2fcdc4d439cbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-bbyje6si/wheels/0e/da/11/c1a678ca18cda6a293502331491891e3c562ef036de57cf602
Successfully built torch-spatiotemporal
  Attempting uninstall: torch-spatiotemporal
    Found existing installation: torch-spatiotemporal 0.9.2
    Uninstalling torch-spatiotemporal-0.9.2:
     

In [ ]:
# Agregarlo como dependencia en el archivo requirements.txt
!pip install kglab

In [ ]:
!pip install geopandas

In [5]:
# from tsl.tsl.datasets import dataobject_prototype
from tsl.datasets import dataobject_prototype

In [6]:
dataset = dataobject_prototype.CrimeMexicoCityTTL()

print(dataset)

--> CURRENT WORK DIRECTORY: [/content]
CURRENT NAMESPASE: 
CrimeMexicoCityTTL(length=12417, n_nodes=10, n_channels=1)


In [ ]:
# Hay que adaptar la info al formato necesario
# 1. Obtener oldest, newest date del dataset
# 2. Conteo a nivel de delegacion por dia y por delito (canal)
# 3. Crear una tabla multiindex donde el indice horizontal son las fechas
#    a nivel de dia, con las delegaciones como indice vertical y
#    el tipo de delito como canal
# 4. Dado que el indice horizontal no tendra datos todos los dias
#    sera necesario rellenar tales dias vacios mediante pd.period_range
# https://stackoverflow.com/questions/47231496/pandas-fill-missing-dates-in-time-series 

In [ ]:
# Organizar la funcion de compute_similarity para considerar delegacion

In [ ]:
# Añadir matrix "covariates"

In [4]:
dataset.get_connectivity(threshold=0.1, include_self=False, normalize_axis=1, layout='edge_index')

AttributeError: ignored

In [25]:
import os
print(f"Current path: {os.getcwd()}")

import numpy as np
import pandas as pd
import kglab
import json
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point, MultiPolygon

def contained(geo_df, points_series):
        # Se obtienen los indices/nombre de las alcaldias a la que pertenecen cada punto
        # el conteo puede llevarse a cabo despues con un built-in method "group-by"
        list_belonging = list(map(geo_df['geometry'].contains, points_series))
        index_geom = [-1 if x.sum() == 0 else geo_df[x].index[0] for x in list_belonging]
        name_geo = ['NA' if x.sum() == 0 else geo_df[x]['nomgeo'].item() for x in list_belonging]
        return index_geom, name_geo

#### La matriz de sitancias entre denuncias y denuncias denuncias las podemos calcular asi
def distance_matrix_crimes(df):
    # #cambiamos la proyección para distancias en metros
    # df = gpd.GeoDataFrame(
    #     df, geometry = gpd.points_from_xy(df.lat, df.long),
    #     crs="EPSG:4326"
    # )
    df = gpd.GeoDataFrame(df, crs='EPSG:4326').to_crs('EPSG:3857')
    # df = df.set_crs('EPSG:4326').to_crs('EPSG:3857')
    dist = df.geometry.apply(lambda g: df.distance(g))
    
    # # Con esta funcion obtenemos la distancia de cada denuncia con respecto de cada centroide
    # return mat
    
    # Save to built directory (Update in the class definition)
    # path = os.path.join(self.root_dir, 'metr_la_dist.npy')
    path = "crime_cdmx_dist.npy"
    np.save(path, dist)


def load_raw():
    # Considero que aqui se va a tener que cargar el TTL y hacer
    # la consulta SPARQL para luego cargarlo a este objeto.
#         self.maybe_build()
#         # load traffic data
#         traffic_path = os.path.join(self.root_dir, 'metr_la.h5')
#         df = pd.read_hdf(traffic_path)
#         # add missing values
#         datetime_idx = sorted(df.index)
#         date_range = pd.date_range(datetime_idx[0],
#                                    datetime_idx[-1],
#                                    freq='5T')
#         df = df.reindex(index=date_range)
#         # load distance matrix
#         path = os.path.join(self.root_dir, 'metr_la_dist.npy')
#         dist = np.load(path)
    NAMESPACES = {
        "wtm":  "http://purl.org/heals/food/",
        "ind":  "http://purl.org/heals/ingredient/",
        "recipe":  "https://www.food.com/recipe/",

        "crime": "http://localhost/ontology2#",
        "cube": "http://purl.org/linked-data/cube#",
        "geo": "http://www.w3.org/2003/01/geo/wgs84_pos#",
        "ontology": "http://dbpedia.org/ontology/",
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    }

    kg = kglab.KnowledgeGraph(namespaces = NAMESPACES)
    print(f"--> CURRENT WORK DIRECTORY: [{os.getcwd()}]")
    _ = kg.load_rdf("/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl")

    # Buscamos tener id, delito, atributos, coordenadas y fecha en el DF
    # A clasificar: crime:tieneCategoria
    # Atributo: crime:contiene, crime:edad, crime:genero
    sparql_contiene = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .   
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:contiene ?atribute .
        }
    """
    sparql_edad = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .   
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:edad ?atribute .
        }
    """
    sparql_genero = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .   
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:genero ?atribute
        }
    """

    df_contiene = kg.query_as_df(sparql=sparql_contiene)
    df_edad = kg.query_as_df(sparql=sparql_edad)
    df_genero = kg.query_as_df(sparql=sparql_genero)

    df_atributes = pd.concat([df_contiene, df_edad, df_genero], ignore_index=True)
    df_atributes = df_atributes.groupby(['uri', 'date', 'type', 'long', 'lat'])['atribute'].apply(list).reset_index(name='atribute')
    # df_atributes
    
    # Luego obtengamos el uri, delito a clasificar
    sparql_crimes = """
        SELECT distinct *
        WHERE {
            ?uri crime:tieneCategoria ?crime .
        }
    """
    df_crimes = kg.query_as_df(sparql=sparql_crimes)
    # Join by uri
    df = pd.merge(df_atributes, df_crimes, how="inner", on="uri")
    # # Let's remove empty lat,long/Invalid rows
    # # valid_rows = (df["long"] != 0.0) & (df["lat"] != 0.0)
    # valid_rows = df[""]
    # df = df[valid_rows]
    # Se le crea la geometria "Point" a cada crimen segun su longitud/latitud
    df["geometry"] = df[["long", "lat"]].T.apply(Point)
    
    # Carga geometrias de alcaldias
    f = open("/content/tsl/tsl/datasets/raw_files_to_remove/alcaldias_cdmx.json", encoding='utf8')
    json_alcaldias = json.load(f)

    ### Mejor usamos directo la funcion de geopandas aunque eso del encoding no jala
    geo_df = gpd.GeoDataFrame.from_features(json_alcaldias["features"])


    # # Encuentra su respectiva alcaldia y las respectivas coordenadas a evaluar
    # if aggregation_level == "alcaldia":
    # Se invoca la funcion y se le agrega la columnas al dataframe
    # return df, geo_df
    index_geom, name_geo = contained(geo_df, df["geometry"])
    df["index_alcaldia"] = index_geom
    df["nombre_alcaldia"] = name_geo
    # Let's remove empty lat,long/Invalid rows
    valid_rows = (df["index_alcaldia"] != -1)
    df = df[valid_rows]

    # # Con esta funcion obtenemos la distancia de cada denuncia con respecto a cada denuncia
    distance_matrix_crimes(df)
    # path = "crime_cdmx_dist.npy"
    # np.save(path, dist)

    # # Carga la matriz de distancias desde el archivo .npy
    # path = os.path.join('', 'crime_cdmx_dist.npy')
    # dist = np.load(path)

    return df, dist

Current path: /content


In [26]:
dft, dist = load_raw()

--> CURRENT WORK DIRECTORY: [/content]


In [33]:
path = os.path.join('', 'crime_cdmx_dist.npy')
dist = np.load(path)
# (dist[-210] == np.Inf).sum()


array([[    0.        , 16364.1972812 , 11106.62506666, ...,
        11713.37346392, 19218.75442714,  5937.86741752],
       [16364.1972812 ,     0.        ,  5727.62988752, ...,
        23189.59567573,  8370.92540425, 21698.6718701 ],
       [11106.62506666,  5727.62988752,     0.        , ...,
        19758.22562267, 12013.57883742, 16794.48393614],
       ...,
       [11713.37346392, 23189.59567573, 19758.22562267, ...,
            0.        , 21549.76892587,  8063.84562384],
       [19218.75442714,  8370.92540425, 12013.57883742, ...,
        21549.76892587,     0.        , 23074.87270322],
       [ 5937.86741752, 21698.6718701 , 16794.48393614, ...,
         8063.84562384, 23074.87270322,     0.        ]])

In [35]:
dist[0]

array([    0.        , 16364.1972812 , 11106.62506666, ...,
       11713.37346392, 19218.75442714,  5937.86741752])

In [24]:
def distance_matrix_crimes(df):
    #cambiamos la proyección para distancias en metros
    df = gpd.GeoDataFrame(
        df, geometry = gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )
    df = df.set_crs('EPSG:4326').to_crs('EPSG:3857')
    dist = df.geometry.apply(lambda g: df.distance(g))

dft10 = dft[:].copy()

gdft = gpd.GeoDataFrame(dft10, crs='EPSG:4326').to_crs('EPSG:3857')
gdft.geometry.apply(lambda g: gdft.distance(g))
# gpd.GeoDataFrame(dft10, geometry = gpd.points_from_xy(dft[:10].long, dft[:10].lat), crs="EPSG:4326")
# 

,0,1,2,3,4,5,6,7,9,10,...,13984,13985,13986,13987,13989,13990,13991,13992,13993,13994
0,0.000000,16364.197281,11106.625067,15784.117603,17470.068525,16621.762500,17974.102145,18662.245619,18201.036320,5825.580419,...,12934.256165,8886.113045,2563.158266,12641.728682,9595.751644,5219.340306,10821.973354,11713.373464,19218.754427,5937.867418
1,16364.197281,0.000000,5727.629888,5034.857950,2165.167672,6321.092568,7985.720802,6530.113484,22485.713515,21335.339564,...,29211.338055,8899.602277,18878.572002,6923.647153,8839.487395,13465.252776,19303.920375,23189.595676,8370.925404,21698.671870
2,11106.625067,5727.629888,0.000000,7854.777878,6415.581414,9255.246957,11115.099856,10580.706571,21428.230166,16518.922849,...,23727.756173,6025.659933,13668.005401,3225.364734,6538.937497,7744.928830,16511.197740,19758.225623,12013.578837,16794.483936
3,15784.117603,5034.857950,7854.777878,0.000000,7166.748699,1447.142116,3312.172212,2955.380645,17894.092243,19758.460841,...,28630.670146,6980.511541,18050.620170,10451.535633,6440.890010,14484.533771,15812.815494,20039.162515,4161.286745,20231.824271
4,17470.068525,2165.167672,6415.581414,7166.748699,0.000000,8400.143013,9981.680995,8348.760465,24624.247044,22699.509906,...,30142.821916,10660.673212,20022.072547,6546.379625,10706.189698,14029.300583,21233.196938,25009.830569,10235.878815,23021.921332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13990,5219.340306,13465.252776,7744.928830,14484.533771,14029.300583,15657.412699,17336.937108,17430.763142,21879.165785,10985.908452,...,16362.978643,8748.380694,7442.767107,8204.034317,9619.315717,0.000000,15033.330710,16680.215898,18465.408558,11020.986068
13991,10821.973354,19303.920375,16511.197740,15812.815494,21233.196938,15707.209195,15983.408518,17698.797176,7577.780105,8932.871192,...,18364.926258,10705.017634,10677.129381,19414.349599,10536.811104,15033.330710,0.000000,4434.634714,17135.443487,9560.817828
13992,11713.373464,23189.595676,19758.225623,20039.162515,25009.830569,20028.771790,20384.422834,22054.232558,9828.458181,7558.403585,...,15646.119961,14350.230003,10568.232937,22395.364589,14350.422910,16680.215898,4434.634714,0.000000,21549.768926,8063.845624
13993,19218.754427,8370.925404,12013.578837,4161.286745,10235.878815,2824.313502,1266.720437,1898.453450,17399.266221,22545.754365,...,31823.809719,10349.069468,21305.171039,14516.797891,9624.008028,18465.408558,17135.443487,21549.768926,0.000000,23074.872703


In [21]:
dft[:10]

,uri,date,type,long,lat,atribute,crime,geometry,index_alcaldia,nombre_alcaldia
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656),4,Iztapalapa
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544),5,Azcapotzalco
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424),2,Venustiano Carranza
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453),10,Miguel Hidalgo
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107),9,Gustavo A. Madero
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123),10,Miguel Hidalgo
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877),10,Miguel Hidalgo
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583),10,Miguel Hidalgo
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891),0,Tlalpan
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074),4,Iztapalapa


In [85]:
dft[:10]

,uri,date,type,long,lat,atribute,crime,geometry
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656)
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544)
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424)
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453)
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107)
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123)
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877)
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583)
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891)
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074)


In [99]:
def contained(geo_df, points_series):
        # Se obtienen los indices/nombre de las alcaldias a la que pertenecen cada punto
        # el conteo puede llevarse a cabo despues con un built-in method "group-by"
        list_belonging = list(map(geo_df['geometry'].contains, points_series))
        # return list_belonging
        index_geom = [-1 if x.sum() == 0 else geo_df[x].index[0] for x in list_belonging]
        name_geo = ['NA' if x.sum() == 0 else geo_df[x]['nomgeo'].item() for x in list_belonging]
        return index_geom, name_geo

# ig, ng = contained(gd, dft["geometry"])
lc = contained(gd, dft["geometry"])

In [101]:
lc[0]

[4,
 5,
 2,
 10,
 9,
 10,
 10,
 10,
 0,
 4,
 9,
 8,
 5,
 10,
 4,
 3,
 9,
 9,
 9,
 9,
 8,
 9,
 3,
 10,
 9,
 6,
 6,
 12,
 0,
 1,
 4,
 9,
 3,
 9,
 4,
 14,
 4,
 9,
 12,
 14,
 9,
 10,
 2,
 3,
 15,
 5,
 0,
 10,
 6,
 0,
 4,
 12,
 4,
 2,
 4,
 5,
 4,
 4,
 2,
 2,
 12,
 8,
 2,
 14,
 11,
 9,
 6,
 10,
 4,
 6,
 2,
 12,
 4,
 8,
 12,
 4,
 9,
 13,
 3,
 6,
 8,
 9,
 9,
 15,
 15,
 8,
 3,
 4,
 6,
 8,
 4,
 8,
 9,
 9,
 12,
 15,
 1,
 9,
 4,
 0,
 3,
 12,
 4,
 6,
 10,
 14,
 9,
 4,
 9,
 10,
 14,
 0,
 4,
 9,
 12,
 14,
 6,
 8,
 4,
 3,
 3,
 4,
 9,
 4,
 6,
 3,
 4,
 3,
 2,
 3,
 3,
 10,
 10,
 3,
 9,
 8,
 10,
 4,
 10,
 10,
 9,
 8,
 7,
 5,
 9,
 4,
 8,
 4,
 4,
 4,
 5,
 14,
 0,
 8,
 6,
 12,
 14,
 3,
 3,
 5,
 3,
 12,
 14,
 8,
 12,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 12,
 14,
 9,
 4,
 14,
 0,
 1,
 4,
 14,
 4,
 4,
 1,
 5,
 10,
 14,
 15,
 14,
 4,
 14,
 2,
 14,
 10,
 3,
 14,
 0,
 9,
 8,
 8,
 10,
 14,
 12,
 9,
 4,
 10,
 3,
 3,
 4,
 9,
 9,
 4,
 5,
 2,
 4,
 10,
 10,
 4,
 4,
 4,
 10,
 3,
 3,
 2,
 6,
 9,
 2,
 14,
 5,
 4,
 6,
 3,
 8,
 12,


In [90]:
dft.iloc[index_fails[:10]]

,uri,date,type,long,lat,atribute,crime,geometry
341,<http://localhost/114499>,2019-03-24,crime:obs,-98.963410,19.314080,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-98.96341 19.31408)
529,<http://localhost/121321>,2018-09-15,crime:obs,-99.220070,19.449890,[<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PU...,<http://localhost/ROBO_A_TRANSEUNTE_EN_VÍA_PÚB...,POINT (-99.22007 19.44989)
3080,<http://localhost/243908>,2019-03-24,crime:obs,-98.963410,19.314080,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-98.96341 19.31408)
4693,<http://localhost/335815>,2020-04-12,crime:obs,-99.219958,19.451691,[<http://localhost/ROBO_A_NEGOCIO_SIN_VIOLENCI...,<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.2199581995967 19.4516911973145)
12986,<http://localhost/889358>,2022-08-30,crime:obs,19.323129,19.323129,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3231287787799 19.3231287787799)
12987,<http://localhost/889494>,2022-08-31,crime:obs,19.304446,19.304446,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3044455697613 19.3044455697613)
12988,<http://localhost/889828>,2022-09-01,crime:obs,19.447017,19.447017,[<http://localhost/NARCOMENUDEO_POSESION_SIMPL...,<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (19.4470166110228 19.4470166110228)
12990,<http://localhost/890013>,2022-09-01,crime:obs,19.366314,19.366314,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (19.3663141575609 19.3663141575609)
12991,<http://localhost/890021>,2022-08-28,crime:obs,19.379062,19.379062,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (19.3790619853489 19.3790619853489)
12993,<http://localhost/890072>,2022-09-01,crime:obs,19.368912,19.368912,[<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PU...,<http://localhost/ROBO_A_TRANSEUNTE_EN_VÍA_PÚB...,POINT (19.3689115621401 19.3689115621401)


In [73]:
s = (dft["long"] != 0.0) & (dft["lat"] != 0.0)
dft[s][:10]

,uri,date,type,long,lat,atribute,crime,geometry
0,<http://localhost/100091>,2019-06-05,crime:obs,-99.07210,19.35656,[<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_M...,POINT (-99.0721 19.35656)
1,<http://localhost/100124>,2019-02-10,crime:obs,-99.14774,19.47544,"[<http://localhost/VIOLENCIA_FAMILIAR>, 38.0, ...",<http://localhost/DELITO_DE_BAJO_IMPACTO>,POINT (-99.14774 19.47544)
2,<http://localhost/10020>,2019-06-02,crime:obs,-99.10834,19.44424,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.10834 19.44424)
3,<http://localhost/100449>,2019-05-18,crime:obs,-99.17890,19.44453,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.1789 19.44453)
4,<http://localhost/100505>,2019-06-07,crime:obs,-99.13757,19.49107,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.13757 19.49107)
5,<http://localhost/100557>,2019-06-07,crime:obs,-99.19142,19.44123,[<http://localhost/ROBO_A_TRANSEUNTE_A_BORDO_D...,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_T...,POINT (-99.19142 19.44123)
6,<http://localhost/100644>,2019-05-28,crime:obs,-99.20802,19.43877,[<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_...,<http://localhost/ROBO_A_CUENTAHABIENTE_SALIEN...,POINT (-99.20802 19.43877)
7,<http://localhost/100665>,2019-03-08,crime:obs,-99.20259,19.45583,[<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCI...,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,POINT (-99.20259 19.45583)
9,<http://localhost/100685>,2019-06-08,crime:obs,-99.22376,19.29891,[<http://localhost/HOMICIDIO_POR_ARMA_DE_FUEGO...,<http://localhost/HOMICIDIO_DOLOSO>,POINT (-99.22376 19.29891)
10,<http://localhost/100728>,2019-06-06,crime:obs,-99.07705,19.30740,[<http://localhost/ROBO_DE_VEHICULO_DE_SERVICI...,<http://localhost/ROBO_DE_VEHÍCULO_CON_Y_SIN_V...,POINT (-99.07705 19.3074)


In [89]:
count = 0
index_fails = []
for n, x in enumerate(lc):
    try:
        gd[x].index[0]
    except:
        # print(x)
        count += 1
        index_fails.append(n)

print("count: ", count)

count:  241


In [97]:
# index_fails
lc[319].sum()

0

In [7]:
NAMESPACES = {
        "wtm":  "http://purl.org/heals/food/",
        "ind":  "http://purl.org/heals/ingredient/",
        "recipe":  "https://www.food.com/recipe/",

        "crime": "http://localhost/ontology2#",
        "cube": "http://purl.org/linked-data/cube#",
        # "geo": "http://www.w3.org/2003/01/geo/wgs84_pos#",
        "ontology": "http://dbpedia.org/ontology/",
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    }

kg = kglab.KnowledgeGraph(namespaces = NAMESPACES)
print(f"--> CURRENT WORK DIRECTORY: [{os.getcwd()}]")
_ = kg.load_rdf("/content/tsl/tsl/datasets/raw_files_to_remove/testind3.ttl")


--> CURRENT WORK DIRECTORY: [/content]


In [10]:
sparql_contiene = """
        SELECT distinct *
        WHERE {
            ?uri a crime:obs .
            ?uri a ?type .   
            ?uri crime:tieneFecha ?date .
            ?uri geo1:lat ?lat .
            ?uri geo1:long ?long .

            ?uri crime:contiene ?atribute .
            # ?uri ?x ?y .
        }
    """
# sparql_contiene = "select distinct * where {?uri a crime:obs .}"
df_contiene = kg.query_as_df(sparql=sparql_contiene)
df_contiene

,atribute,uri,date,lat,long,type
0,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_P...,<http://localhost/100091>,2019-06-05,19.35656,-99.07210,crime:obs
1,<http://localhost/VIOLENCIA_FAMILIAR>,<http://localhost/100124>,2019-02-10,19.47544,-99.14774,crime:obs
2,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,<http://localhost/10020>,2019-06-02,19.44424,-99.10834,crime:obs
3,<http://localhost/ROBO_A_NEGOCIO_CON_VIOLENCIA>,<http://localhost/100449>,2019-05-18,19.44453,-99.17890,crime:obs
4,<http://localhost/ROBO_DE_VEHICULO_DE_SERVICIO...,<http://localhost/100505>,2019-06-07,19.49107,-99.13757,crime:obs
...,...,...,...,...,...,...
13995,<http://localhost/ROBO_A_PASAJERO_A_BORDO_DE_P...,<http://localhost/99795>,2019-10-05,19.39879,-99.05816,crime:obs
13996,<http://localhost/ROBO_A_TRANSEUNTE_EN_VIA_PUB...,<http://localhost/99803>,2019-10-05,19.31211,-99.15714,crime:obs
13997,<http://localhost/VIOLACION>,<http://localhost/99881>,2019-10-06,19.27914,-99.13799,crime:obs
13998,<http://localhost/ROBO_A_TRANSEUNTE_SALIENDO_D...,<http://localhost/9992>,2019-06-03,19.44619,-99.21624,crime:obs


In [4]:
kg.describe_ns()

,prefix,namespace
0,dct,http://purl.org/dc/terms/
1,prov,http://www.w3.org/ns/prov#
2,schema,https://schema.org/
3,sh,http://www.w3.org/ns/shacl#
4,skos,http://www.w3.org/2004/02/skos/core#
5,wtm,http://purl.org/heals/food/
6,ind,http://purl.org/heals/ingredient/
7,recipe,https://www.food.com/recipe/
8,crime,http://localhost/ontology2#
9,cube,http://purl.org/linked-data/cube#
